In [1]:
import math
import re
import sys
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve,roc_auc_score

# device =torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:0')
device = torch.device('cpu')

In [4]:
ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-10.csv')
# ori_fea = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\初始transformer所有特征-45.csv')

ori_fea = ori_fea.drop(ori_fea.columns[0], axis=1)

label = [0] * 8000 + [1] * 8000

oy = pd.DataFrame(label, columns=['label'])

ori_fea

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.380952,315,0.346032,0.155556,0.225397,0.273016,0.169329,0.015974,0.089457,0.070288,...,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654
1,0.409396,596,0.261745,0.177852,0.231544,0.328859,0.104377,0.042088,0.055556,0.060606,...,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654
2,0.343101,819,0.312576,0.191697,0.151404,0.344322,0.105263,0.059976,0.048960,0.097919,...,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654
3,0.461774,327,0.244648,0.299694,0.162080,0.293578,0.080000,0.058462,0.036923,0.067692,...,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654
4,0.565217,23,0.130435,0.347826,0.217391,0.304348,0.047619,0.000000,0.000000,0.095238,...,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.450256,975,0.274872,0.217436,0.232821,0.274872,0.069887,0.045221,0.073998,0.085303,...,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654,2.203654
15996,0.430957,1557,0.289017,0.185613,0.245344,0.280026,0.093248,0.041801,0.079100,0.074598,...,1.222891,3.279911,2.326407,3.279911,1.965121,2.326407,1.965121,1.965121,1.222891,1.965121
15997,0.455010,2934,0.311179,0.190184,0.264826,0.233810,0.106753,0.042633,0.103342,0.058322,...,1.222891,1.965121,3.279911,1.965121,2.326407,3.279911,1.965121,2.326407,1.965121,1.965121
15998,0.448404,2694,0.293615,0.213066,0.235338,0.257981,0.089525,0.056092,0.080609,0.067608,...,1.222891,1.222891,1.222891,2.326407,3.279911,1.965121,1.965121,1.222891,1.222891,1.965121


In [5]:
# fasta to csv (extract the sequences)
pattern_title = re.compile(r'^>.*', re.M)
pattern_n = re.compile(r'\n')

with open(r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\cRNA\Musa acuminata.fasta', 'r', encoding='utf-8') as f:
    text = f.read()

data = re.split(pattern_title, text)
data = [re.sub(pattern_n, '', i) for i in data]
data = pd.DataFrame(data[1:])
data.columns = ['seq']
data = data[:1984]

with open(r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\ncRNA\Musa acuminata.fasta', 'r', encoding='utf-8') as f:
    text = f.read()

data_nc = re.split(pattern_title, text)
data_nc = [re.sub(pattern_n, '', i) for i in data_nc]
data_nc = pd.DataFrame(data_nc[1:])
data_nc.columns = ['seq']
data_nc = data_nc[:1984]

data = pd.concat([data,data_nc]).reset_index(drop=True)

# 创建一个包含4000个元素的列表，前2000个为0，后2000个为1
label = [0] * 1984 + [1] * 1984

# 使用pandas库将列表转换为DataFrame
y = pd.DataFrame(label, columns=['label'])

data.insert(0,'label',y)
data3 = data.copy()
data

,label,seq
0,0,ATGAAGAGGAATAGACTTGGAGGAGGAGGAGGAGGAGGAGAGATGG...
1,0,ATGACGCAGGCCGTCGGCCGGGACCCCCGCATGGGCGCCTCCATCC...
2,0,ATGACAACTGAAGAGCTTGAAAATATTTTGAGAATGCAACACGCTG...
3,0,ATGATCATAGAGTTTAAACATTGTGTGCTGACGGCCATTTGTGGTT...
4,0,ATGACCCGCGCTTCTCTCCACCCCGGCCGCTTCACCCTCCCCTTCC...
...,...,...
3963,1,GAAGAAGATGAGGATGATGCAGAAGATGGCGATTCAAAGTATGATT...
3964,1,ATGGAGGTGAAGGCAACAGCATGTGCTCTCCTCATGTTTCTGCTTT...
3965,1,ATGACCCTTGCAACCAATCTACTCCCTAATGTTTCCATGAAGTCAG...
3966,1,ATGCATGAAATACAACCTCGAGGAGGCGGGCGAAGTCAAGTAACCT...


## 手动特征

In [6]:
from itertools import product
# compute k-mers features (k=1~4)
def Kmers_funct(seq, x):
    X = [None] * len(seq)
    for i in range(len(seq)):
        a = seq[i]
        t = 0
        l = []
        for index in range(len(a) - x + 1):
            t = a[index:index + x]
            if (len(t)) == x:
                l.append(t)
        X[i] = l
    return X


def nucleotide_type(k):
    z = []
    for i in product('ACGT', repeat=k):
        z.append(''.join(i))
    return z


def Kmers_frequency(seq, x):
    X = []
    char = nucleotide_type(x)
    for i in range(len(seq)):
        s = seq[i]
        frequence = []
        for a in char:
            number = s.count(a)
            char_frequence = number / (len(s) - x + 1)
            frequence.append(char_frequence)
        X.append(frequence)
    return X


feature_1mer = Kmers_funct(data.seq, 1)  # 1-mer
feature_2mer = Kmers_funct(data.seq, 2)  # 2-mer
feature_3mer = Kmers_funct(data.seq, 3)  # 3-mer

feature_1mer_frequency = Kmers_frequency(feature_1mer, 1)  # 1-mer
feature_2mer_frequency = Kmers_frequency(feature_2mer, 2)  # 2-mer
feature_3mer_frequency = Kmers_frequency(feature_3mer, 3)  # 3-mer

feature = pd.concat([pd.DataFrame(feature_1mer_frequency),
                     pd.DataFrame(feature_2mer_frequency),
                     pd.DataFrame(feature_3mer_frequency)], axis=1)
feature.columns = ['A','C','G','T','AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT','AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']
feature

,A,C,G,T,AA,AC,AG,AT,CA,CC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.247525,0.257426,0.353135,0.141914,0.056478,0.043189,0.112957,0.033223,0.069767,0.076412,...,0.020067,0.006689,0.010033,0.016722,0.020067,0.010033,0.000000,0.016722,0.010033,0.000000
1,0.167358,0.366528,0.315353,0.150761,0.029126,0.069348,0.037448,0.030513,0.058252,0.110957,...,0.026426,0.009736,0.012517,0.018081,0.015299,0.002782,0.002782,0.011127,0.002782,0.001391
2,0.256225,0.249799,0.213655,0.280321,0.071601,0.048270,0.061142,0.074819,0.073210,0.061947,...,0.005641,0.026591,0.020951,0.030620,0.019339,0.013699,0.010475,0.018533,0.025786,0.017728
3,0.224638,0.224638,0.304348,0.246377,0.032847,0.058394,0.058394,0.072993,0.058394,0.036496,...,0.044118,0.011029,0.018382,0.007353,0.025735,0.018382,0.007353,0.014706,0.025735,0.007353
4,0.230141,0.246474,0.268745,0.254640,0.052045,0.041636,0.066171,0.069888,0.066171,0.072119,...,0.009680,0.012658,0.019360,0.021593,0.026061,0.022338,0.008191,0.014892,0.028295,0.014892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,0.321168,0.160584,0.240876,0.277372,0.084559,0.040441,0.110294,0.088235,0.066176,0.029412,...,0.007407,0.003704,0.033333,0.018519,0.011111,0.011111,0.018519,0.029630,0.022222,0.040741
3964,0.168831,0.285714,0.320346,0.225108,0.026201,0.034934,0.065502,0.039301,0.065502,0.069869,...,0.000000,0.035242,0.022026,0.052863,0.013216,0.022026,0.000000,0.022026,0.004405,0.013216
3965,0.380952,0.242857,0.104762,0.271429,0.149038,0.076923,0.043269,0.110577,0.100962,0.076923,...,0.000000,0.014563,0.024272,0.009709,0.000000,0.009709,0.019417,0.024272,0.004854,0.024272
3966,0.271111,0.235556,0.257778,0.235556,0.103139,0.049327,0.062780,0.053812,0.040359,0.049327,...,0.009050,0.022624,0.022624,0.022624,0.018100,0.004525,0.009050,0.013575,0.022624,0.022624


In [7]:
def gc_content(dna_sequence):
    gc_percent = []
    for i in dna_sequence:
        gc_count = i.count('G') + i.count('C')
        total_count = len(i)
        gc_percent_eve = (gc_count / total_count) * 100
        gc_percent.append(gc_percent_eve)
    return gc_percent

In [8]:
# gc_precent = gc_content(data.iloc[:,0])
gc_precent = gc_content(data.seq)
gc_precent_pd = pd.DataFrame(gc_precent)
gc_precent_pd.rename(columns={0: 'GC content'}, inplace=True)
print(gc_precent_pd.shape)
gc_precent_pd

feature.insert(0,'GC content',gc_precent_pd)

feature = pd.DataFrame(feature)
feature.iloc[:,0] = feature.iloc[:,0]/100
feature

(3968, 1)


,GC content,A,C,G,T,AA,AC,AG,AT,CA,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.610561,0.247525,0.257426,0.353135,0.141914,0.056478,0.043189,0.112957,0.033223,0.069767,...,0.020067,0.006689,0.010033,0.016722,0.020067,0.010033,0.000000,0.016722,0.010033,0.000000
1,0.681881,0.167358,0.366528,0.315353,0.150761,0.029126,0.069348,0.037448,0.030513,0.058252,...,0.026426,0.009736,0.012517,0.018081,0.015299,0.002782,0.002782,0.011127,0.002782,0.001391
2,0.463454,0.256225,0.249799,0.213655,0.280321,0.071601,0.048270,0.061142,0.074819,0.073210,...,0.005641,0.026591,0.020951,0.030620,0.019339,0.013699,0.010475,0.018533,0.025786,0.017728
3,0.528986,0.224638,0.224638,0.304348,0.246377,0.032847,0.058394,0.058394,0.072993,0.058394,...,0.044118,0.011029,0.018382,0.007353,0.025735,0.018382,0.007353,0.014706,0.025735,0.007353
4,0.515219,0.230141,0.246474,0.268745,0.254640,0.052045,0.041636,0.066171,0.069888,0.066171,...,0.009680,0.012658,0.019360,0.021593,0.026061,0.022338,0.008191,0.014892,0.028295,0.014892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,0.401460,0.321168,0.160584,0.240876,0.277372,0.084559,0.040441,0.110294,0.088235,0.066176,...,0.007407,0.003704,0.033333,0.018519,0.011111,0.011111,0.018519,0.029630,0.022222,0.040741
3964,0.606061,0.168831,0.285714,0.320346,0.225108,0.026201,0.034934,0.065502,0.039301,0.065502,...,0.000000,0.035242,0.022026,0.052863,0.013216,0.022026,0.000000,0.022026,0.004405,0.013216
3965,0.347619,0.380952,0.242857,0.104762,0.271429,0.149038,0.076923,0.043269,0.110577,0.100962,...,0.000000,0.014563,0.024272,0.009709,0.000000,0.009709,0.019417,0.024272,0.004854,0.024272
3966,0.493333,0.271111,0.235556,0.257778,0.235556,0.103139,0.049327,0.062780,0.053812,0.040359,...,0.009050,0.022624,0.022624,0.022624,0.018100,0.004525,0.009050,0.013575,0.022624,0.022624


In [9]:
sequence_len = []
for i in range(data.shape[0]):
    sequence_len.append(len(data.seq[i]))
sequence_len = pd.DataFrame(sequence_len)
sequence_len.rename(columns={0:'seq_len'},inplace=True)
sequence_len

feature.insert(1,'seq_len',sequence_len)
feature.rename(columns={'GC content':'GC_con'},inplace=True)
print(feature.shape)
feature

(3968, 86)


,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
0,0.610561,303,0.247525,0.257426,0.353135,0.141914,0.056478,0.043189,0.112957,0.033223,...,0.020067,0.006689,0.010033,0.016722,0.020067,0.010033,0.000000,0.016722,0.010033,0.000000
1,0.681881,723,0.167358,0.366528,0.315353,0.150761,0.029126,0.069348,0.037448,0.030513,...,0.026426,0.009736,0.012517,0.018081,0.015299,0.002782,0.002782,0.011127,0.002782,0.001391
2,0.463454,1245,0.256225,0.249799,0.213655,0.280321,0.071601,0.048270,0.061142,0.074819,...,0.005641,0.026591,0.020951,0.030620,0.019339,0.013699,0.010475,0.018533,0.025786,0.017728
3,0.528986,276,0.224638,0.224638,0.304348,0.246377,0.032847,0.058394,0.058394,0.072993,...,0.044118,0.011029,0.018382,0.007353,0.025735,0.018382,0.007353,0.014706,0.025735,0.007353
4,0.515219,1347,0.230141,0.246474,0.268745,0.254640,0.052045,0.041636,0.066171,0.069888,...,0.009680,0.012658,0.019360,0.021593,0.026061,0.022338,0.008191,0.014892,0.028295,0.014892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,0.401460,274,0.321168,0.160584,0.240876,0.277372,0.084559,0.040441,0.110294,0.088235,...,0.007407,0.003704,0.033333,0.018519,0.011111,0.011111,0.018519,0.029630,0.022222,0.040741
3964,0.606061,231,0.168831,0.285714,0.320346,0.225108,0.026201,0.034934,0.065502,0.039301,...,0.000000,0.035242,0.022026,0.052863,0.013216,0.022026,0.000000,0.022026,0.004405,0.013216
3965,0.347619,210,0.380952,0.242857,0.104762,0.271429,0.149038,0.076923,0.043269,0.110577,...,0.000000,0.014563,0.024272,0.009709,0.000000,0.009709,0.019417,0.024272,0.004854,0.024272
3966,0.493333,225,0.271111,0.235556,0.257778,0.235556,0.103139,0.049327,0.062780,0.053812,...,0.009050,0.022624,0.022624,0.022624,0.018100,0.004525,0.009050,0.013575,0.022624,0.022624


# cds特征

In [10]:
names = ['seqname', 'start', 'end', 'source ', 'accession', 'score ', 'startComplete ',
                                 'endComplete ', 'cdsCount ', 'cdsStarts ', 'cdsSizes ']
index_col = "seqname"

rna_cds = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\c_cds_test\Musa_acuminata',names=names,sep='\t', index_col=index_col)
rna_cds = rna_cds[:1984]
rna_cds_nc = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\\ncRNA_test\\nc_cds_test\Musa_acuminata',names=names,sep='\t', index_col=index_col)
rna_cds_nc = rna_cds_nc[:1984]

rna_cds = pd.concat([rna_cds,rna_cds_nc]).reset_index(drop=True)
rna_cds

data1 = data
data2 = data
for i, l in data2.iterrows():
    data1.iloc[i,1] = len(l['seq'])
    
mylist = []
mylist = pd.DataFrame(mylist,columns=['score','cdsStarts','cdsStop','cdsSizes','cdsPercent'])

for index, row in rna_cds.iterrows():
    tem_pd_score = pd.DataFrame(row)
    mylist.loc[index, 'cdsStarts'] = row["start"]
    mylist.loc[index, 'cdsStop'] = row["end"]
    mylist.loc[index, 'cdsSizes'] = row["end"] - row["start"]
    mylist.loc[index, 'score'] = tem_pd_score.iloc[4, 0]
    mylist.loc[index, 'cdsPercent'] = (row["start"] + row["end"]) / data1.loc[index, 'seq']
    
feature = feature.join(mylist)

# 选取其中10个特征
# feature = feature[['GC_con', 'score', 'cdsStop', 'cdsSizes','T','C','GT','GC','ACG','TAT']]
# 选取其中45个特征
# feature = feature[['GC_con', 'seq_len', 'A', 'C', 'G', 'T', 'AA', 'AC', 'AG', 'AT', 'CA', 'CC', 'CG', 'CT', 'GA', 'GC', 'GG', 'GT', 'TA', 'TC', 'TG', 'TT', 'AAA', 'AAG', 'AGA', 'ATT', 'CCG', 'CGC', 'CGG', 'CTC', 'GAA', 'GAG', 'GCC', 'GCG', 'GGC', 'GGG', 'TCT', 'TGG', 'TTG', 'TTT', 'score', 'cdsStarts', 'cdsStop', 'cdsSizes', 'cdsPercent']]

feature

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,TGT,TTA,TTC,TTG,TTT,score,cdsStarts,cdsStop,cdsSizes,cdsPercent
0,0.610561,303,0.247525,0.257426,0.353135,0.141914,0.056478,0.043189,0.112957,0.033223,...,0.010033,0.000000,0.016722,0.010033,0.000000,440.0,42,303,261,1.138614
1,0.681881,723,0.167358,0.366528,0.315353,0.150761,0.029126,0.069348,0.037448,0.030513,...,0.002782,0.002782,0.011127,0.002782,0.001391,878.0,30,723,693,1.041494
2,0.463454,1245,0.256225,0.249799,0.213655,0.280321,0.071601,0.048270,0.061142,0.074819,...,0.013699,0.010475,0.018533,0.025786,0.017728,1395.5,33,1245,1212,1.026506
3,0.528986,276,0.224638,0.224638,0.304348,0.246377,0.032847,0.058394,0.058394,0.072993,...,0.018382,0.007353,0.014706,0.025735,0.007353,376.0,0,276,276,1.0
4,0.515219,1347,0.230141,0.246474,0.268745,0.254640,0.052045,0.041636,0.066171,0.069888,...,0.022338,0.008191,0.014892,0.028295,0.014892,1447.0,0,1347,1347,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,0.401460,274,0.321168,0.160584,0.240876,0.277372,0.084559,0.040441,0.110294,0.088235,...,0.011111,0.018519,0.029630,0.022222,0.040741,326.0,7,133,126,0.510949
3964,0.606061,231,0.168831,0.285714,0.320346,0.225108,0.026201,0.034934,0.065502,0.039301,...,0.022026,0.000000,0.022026,0.004405,0.013216,431.0,0,231,231,1.0
3965,0.347619,210,0.380952,0.242857,0.104762,0.271429,0.149038,0.076923,0.043269,0.110577,...,0.009709,0.019417,0.024272,0.004854,0.024272,310.0,0,210,210,1.0
3966,0.493333,225,0.271111,0.235556,0.257778,0.235556,0.103139,0.049327,0.062780,0.053812,...,0.004525,0.009050,0.013575,0.022624,0.022624,325.0,0,225,225,1.0


## 深度特征

In [11]:
# data = pd.read_csv(r'C:\Users\26970\Desktop\特征融合\基准数据集.csv')  # 导入数据
sequences = data3['seq']  # 序列
labels = data3['label'].values  # 标签

pat = re.compile('[AGCTagct]')

def pre_process(text):
    text = pat.findall(text)
    text = [each.lower() for each in text]
    return text

x = sequences.apply(pre_process)

#创建词典
word_set = set()  # 初始化

for lst in x:
    for word in lst:
        word_set.add(word)

word_list = list(word_set)
word_index = dict([(each, word_list.index(each) + 1) for each in word_list])

#将序列中的字母转为数字
text = x.apply(lambda x: [word_index.get(word, 0) for word in x])

#固定序列的长度，进行截断或填充操作
text_len = 1200  # 长度固定为1000
pad_text = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text]  # 用0填充或者截断
pad_text = np.array(pad_text)  # 转为数组

#划分训练集和测试集
pad_text,labels=torch.LongTensor(pad_text),torch.LongTensor(labels)
#x_train, x_test, y_train, y_test = train_test_split(pad_text, labels, test_size=0.3)  # 默认采用分层抽样构建训练集


#构建数据迭代器
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, text_list, label_list):  # 将序列和标签保存在类属性中
        self.text_list = text_list
        self.label_list = label_list

    def __getitem__(self, index):  # 索引序列和标签
        text = torch.LongTensor(self.text_list[index])  # Tensor中的元素转换为64位整型
        label = self.label_list[index]
        return text, label

    def __len__(self):
        return len(self.text_list)  # 返回序列数

data_ds = Mydataset(pad_text,labels)
#train_ds = Mydataset(x_train, y_train)  # 将训练集单独存储
#test_ds = Mydataset(x_test, y_test)  # 将测试集单独存储

batch_size = 64  # 批量大小

data_dl =torch.utils.data.DataLoader(data_ds,batch_size=batch_size,shuffle=False)
#train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
#test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

#定义网络架构
embed_dim = 24  # 嵌入维度
hidden_size = 20  # 隐藏层单元数


class Net(nn.Module):
    def __init__(self, word_list, embed_dim, hidden_size):
        super(Net, self).__init__()
        self.em = nn.Embedding(len(word_list) + 1, embed_dim)
        self.rnn = nn.LSTM(embed_dim, hidden_size, bidirectional=True)
        self.pool1 = nn.MaxPool1d(5)
        self.pool2 = nn.MaxPool1d(3)
        self.fc1 = nn.Linear(2 * hidden_size, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, inputs):
        bz = inputs.shape[1]
        h0 = torch.zeros((2, bz, hidden_size)).to(device)
        c0 = torch.zeros((2, bz, hidden_size)).to(device)
        x = self.em(inputs)
        r_o, _ = self.rnn(x, (h0, c0))
        x = self.pool1(x)
        x = self.pool2(x)
        x = x.view(64, -1)
        #r_o = r_o[-1]
        #x = F.dropout(F.relu(self.fc1(r_o)))
        #x = self.fc2(x)

        return x

model = Net(word_list, embed_dim, hidden_size)  # 实例化
model = model.to(device)  

loss = nn.CrossEntropyLoss()  # 默认求每个batch下的平均损失
loss = loss.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

with torch.no_grad():
    outputs=[]
    for x,y in data_dl:
        out = model(x)
        outputs.append(out)
#     print(outputs)
#     print(outputs[0].shape)
#     print(len(outputs))
#r1 = [i.reshape(16000,1) for i in out]
outputs1 = torch.concat(outputs,axis =0)
# print(outputs1)

deep_fea=pd.DataFrame(outputs1)
deep_fea
deep_fea.to_csv(r'C:\Users\26970\Desktop\特征融合\91特征\Cicer arietinum-bilstm.csv')

In [12]:
fea_all = feature.join(deep_fea)
fea_all

,GC_con,seq_len,A,C,G,T,AA,AC,AG,AT,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0.610561,303,0.247525,0.257426,0.353135,0.141914,0.056478,0.043189,0.112957,0.033223,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
1,0.681881,723,0.167358,0.366528,0.315353,0.150761,0.029126,0.069348,0.037448,0.030513,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
2,0.463454,1245,0.256225,0.249799,0.213655,0.280321,0.071601,0.048270,0.061142,0.074819,...,1.851674,1.851674,2.390463,1.851674,1.851674,1.593740,2.390463,1.851674,2.047556,1.593740
3,0.528986,276,0.224638,0.224638,0.304348,0.246377,0.032847,0.058394,0.058394,0.072993,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
4,0.515219,1347,0.230141,0.246474,0.268745,0.254640,0.052045,0.041636,0.066171,0.069888,...,1.851674,2.047556,2.390463,2.047556,1.851674,2.047556,2.390463,2.047556,2.390463,2.047556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,0.401460,274,0.321168,0.160584,0.240876,0.277372,0.084559,0.040441,0.110294,0.088235,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
3964,0.606061,231,0.168831,0.285714,0.320346,0.225108,0.026201,0.034934,0.065502,0.039301,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
3965,0.347619,210,0.380952,0.242857,0.104762,0.271429,0.149038,0.076923,0.043269,0.110577,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
3966,0.493333,225,0.271111,0.235556,0.257778,0.235556,0.103139,0.049327,0.062780,0.053812,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371


In [13]:
y = pd.DataFrame(labels, columns=['label'])

fea_all.insert(0,'label',y)
fea_all

,label,GC_con,seq_len,A,C,G,T,AA,AC,AG,...,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199
0,0,0.610561,303,0.247525,0.257426,0.353135,0.141914,0.056478,0.043189,0.112957,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
1,0,0.681881,723,0.167358,0.366528,0.315353,0.150761,0.029126,0.069348,0.037448,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
2,0,0.463454,1245,0.256225,0.249799,0.213655,0.280321,0.071601,0.048270,0.061142,...,1.851674,1.851674,2.390463,1.851674,1.851674,1.593740,2.390463,1.851674,2.047556,1.593740
3,0,0.528986,276,0.224638,0.224638,0.304348,0.246377,0.032847,0.058394,0.058394,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
4,0,0.515219,1347,0.230141,0.246474,0.268745,0.254640,0.052045,0.041636,0.066171,...,1.851674,2.047556,2.390463,2.047556,1.851674,2.047556,2.390463,2.047556,2.390463,2.047556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3963,1,0.401460,274,0.321168,0.160584,0.240876,0.277372,0.084559,0.040441,0.110294,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
3964,1,0.606061,231,0.168831,0.285714,0.320346,0.225108,0.026201,0.034934,0.065502,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
3965,1,0.347619,210,0.380952,0.242857,0.104762,0.271429,0.149038,0.076923,0.043269,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371
3966,1,0.493333,225,0.271111,0.235556,0.257778,0.235556,0.103139,0.049327,0.062780,...,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371,2.967371


In [14]:
# 定义朴素贝叶斯模型
from sklearn.metrics import accuracy_score, f1_score, auc, matthews_corrcoef, precision_score, recall_score, confusion_matrix, roc_curve
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_predict

def model_gnb(X, y, X_test, y_test):
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    gnb = GaussianNB()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行五倍交叉验证
    cv_results = cross_validate(gnb, X, y, scoring=scoring, cv=5, return_train_score=False)

    # 计算并输出评估指标
    # print("Accuracy: %f (+/- %f)" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std() * 2))
    # print("F1 score: %f (+/- %f)" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std() * 2))
    # print("ROC AUC score: %f (+/- %f)" % (cv_results['test_roc_auc'].mean(), cv_results['test_roc_auc'].std() * 2))
    # print("MCC: %f (+/- %f)" % (cv_results['test_mcc'].mean(), cv_results['test_mcc'].std() * 2))

    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(gnb, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    gnb.fit(X, y)
    y_pred_test = gnb.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_gnb(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Accuracy: 0.782625
F1 score: 0.743736
ROC AUC score: 0.914287
MCC: 0.594332


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

NPV: 0.717697
PPV: 0.902743
Sensitivity (Recall): 0.633500
Specificity: 0.931750


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Set - Accuracy: 0.655494
Test Set - F1 score: 0.735436
Test Set - ROC AUC score: 0.655494
Test Set - MCC: 0.390330
Test Set - NPV: 0.892994
Test Set - PPV: 0.596921
Test Set - Sensitivity (Recall): 0.957661
Test Set - Specificity: 0.353327


In [21]:
# 定义SVM模型

def model_SVM(X, y, X_test, y_test):
    from sklearn.svm import SVC
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict
    
    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建SVM分类器
    svm_clf = SVC()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(svm_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(svm_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")

    # 在独立测试集上评估模型
    svm_clf.fit(X, y)
    y_pred_test = svm_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")
    
model_SVM(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

Accuracy: 0.928125
F1 score: 0.922393
ROC AUC score: 0.967393
MCC: 0.865402


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

NPV: 0.874317
PPV: 1.000000
Sensitivity (Recall): 0.856250
Specificity: 1.000000


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Test Set - Accuracy: 0.078629
Test Set - F1 score: 0.000000
Test Set - ROC AUC score: 0.078629
Test Set - MCC: -0.853360
Test Set - NPV: 0.135889
Test Set - PPV: 0.000000
Test Set - Sensitivity (Recall): 0.000000
Test Set - Specificity: 0.157258


In [17]:
### RFC训练模型

def model_RFC(X, y, X_test, y_test):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import cross_validate
    from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
    from sklearn.model_selection import cross_val_predict

    X.columns = X.columns.astype(str)
    X_test.columns = X_test.columns.astype(str)
    
    # 创建随机森林分类器
    rf_clf = RandomForestClassifier()

    # 定义评估指标
    scoring = {'accuracy': 'accuracy',
           'f1': 'f1',
           'roc_auc': 'roc_auc',
           'mcc': make_scorer(matthews_corrcoef)}

    # 进行5倍交叉验证并计算评估指标
    cv_results = cross_validate(rf_clf, X, y, cv=5, scoring=scoring)

    # 输出评估指标的平均值和标准差
    print("Accuracy: %f" % (cv_results['test_accuracy'].mean()))
    print("F1 score: %f" % (cv_results['test_f1'].mean()))
    print("ROC AUC score: %f" % (cv_results['test_roc_auc'].mean()))
    print("MCC: %f" % (cv_results['test_mcc'].mean()))

    # 计算混淆矩阵
    y_pred = cross_val_predict(rf_clf, X, y, cv=5)
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    npv = tn / (tn + fn)
    ppv = tp / (tp + fp)
    se = tp / (tp + fn)
    spc = tn / (tn + fp)

    # 输出混淆矩阵及相关指标
    print(f"NPV: {npv:f}")
    print(f"PPV: {ppv:f}")
    print(f"Sensitivity (Recall): {se:f}")
    print(f"Specificity: {spc:f}")
    
    # 在独立测试集上评估模型
    rf_clf.fit(X, y)
    y_pred_test = rf_clf.predict(X_test)
    auc_score_test = roc_auc_score(y_test, y_pred_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_score_test = f1_score(y_test, y_pred_test)
    mcc_test = matthews_corrcoef(y_test, y_pred_test)

    # 输出独立测试集上的评估指标
    print("Test Set - Accuracy: %f" % (accuracy_test))
    print("Test Set - F1 score: %f" % (f1_score_test))
    print("Test Set - ROC AUC score: %f" % (auc_score_test))
    print("Test Set - MCC: %f" % (mcc_test))
    
    # 计算混淆矩阵和评估指标（独立测试集）
    tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    npv_test = tn_test / (tn_test + fn_test)
    ppv_test = tp_test / (tp_test + fp_test)
    se_test = tp_test / (tp_test + fn_test)
    spc_test = tn_test / (tn_test + fp_test)

    # 输出混淆矩阵及相关指标（独立测试集）
    print(f"Test Set - NPV: {npv_test:f}")
    print(f"Test Set - PPV: {ppv_test:f}")
    print(f"Test Set - Sensitivity (Recall): {se_test:f}")
    print(f"Test Set - Specificity: {spc_test:f}")


model_RFC(ori_fea,oy,fea_all.iloc[:,1:],fea_all.iloc[:,0])

D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarni

Accuracy: 0.939813
F1 score: 0.936316
ROC AUC score: 0.964674
MCC: 0.884888


D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
D:\toolsMore\Anaconda\anaconda\lib\site-packages\sklearn\model_selection\_validation.py:1068: DataConversionW

NPV: 0.896280
PPV: 0.992987
Sensitivity (Recall): 0.885000
Specificity: 0.993750


C:\Users\26970\AppData\Local\Temp\ipykernel_20860\235912778.py:46: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_clf.fit(X, y)


Test Set - Accuracy: 0.119708
Test Set - F1 score: 0.000000
Test Set - ROC AUC score: 0.119708
Test Set - MCC: -0.783367
Test Set - NPV: 0.193168
Test Set - PPV: 0.000000
Test Set - Sensitivity (Recall): 0.000000
Test Set - Specificity: 0.239415
